# Train YOLOv2 on LISA Dataset
-----------------------------------------------


In [1]:
import sys
print(sys.version) # Check Python Version

2.7.13 |Anaconda 4.4.0 (64-bit)| (default, Dec 20 2016, 23:09:15) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


In [2]:
import numpy as np
import os
from keras.optimizers import Adam
from utils.parse_input import load_data    # Data handler for LISA dataset
from cfg import *

Using TensorFlow backend.


### Prepare LISA Dataset

In [3]:
lisa_path = "/home/ubuntu/dataset/training/" # Remember the `/` at the end
pretrained_path = "/home/ubuntu/dataset/darknet19_544.weights"
#from convert_lisa_to_txt import save_lisa_to_txt
# save_lisa_to_txt(lisa_path)

In [4]:
x_train, y_train = load_data('training.txt')
labels           = np.unique(y_train[:,1])
num_classes      = len(labels)            # Count number of classes in the dataset

print("Train: {} samples\nNumber of classes: {}".format(len(x_train),num_classes))

Number of ground truth boxes: 3672 boxes
Train: 3672 samples
Number of classes: 31


### Construct YOLOv2 On Keras

In [5]:
from yolov2.model import YOLOv2, darknet19

darknet19 = darknet19(pretrained_path, freeze_layers=True)
yolov2    = YOLOv2(feature_extractor=darknet19, num_anchors=len(ANCHORS), num_classes=N_CLASSES)
model     = yolov2.model

# HYPER-PARAMETERS
BATCH_SIZE = 8
EPOCHS     = 5
LEARN_RATE = 0.01

print(model.summary())

Pre-trained weights have been loaded into model
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________

____________________________________________________________________________________________________
batch_normalization_16 (BatchNor (None, None, None, 10 4096        conv2d_16[0][0]                  
____________________________________________________________________________________________________
leaky_re_lu_16 (LeakyReLU)       (None, None, None, 10 0           batch_normalization_16[0][0]     
____________________________________________________________________________________________________
conv2d_17 (Conv2D)               (None, None, None, 51 524288      leaky_re_lu_16[0][0]             
____________________________________________________________________________________________________
batch_normalization_17 (BatchNor (None, None, None, 51 2048        conv2d_17[0][0]                  
____________________________________________________________________________________________________
leaky_re_lu_17 (LeakyReLU)       (None, None, None, 51 0           batch_normalization_17[0

In [6]:
# Compile and train model
# from yolov2.loss import yolo_loss
import keras
from utils.data_generator import flow_from_list
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

train_data_gen = flow_from_list(X_train, Y_train, ANCHORS, batch_size=BATCH_SIZE)
val_data_gen   = flow_from_list(X_test,  Y_test,ANCHORS, batch_size=BATCH_SIZE, augment_data=False)

# For Debugging purpose
tf_board = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
check_pt = keras.callbacks.ModelCheckpoint('models/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
                                           monitor='val_loss', 
                                           verbose=0, save_best_only=False, 
                                           save_weights_only=False, mode='auto', period=1)

img, labels = val_data_gen.next()
print(np.shape(labels))

(8, 40, 30, 5, 32)


In [9]:
import keras.backend as K
import tensorflow as tf

def custom_loss(y_true, y_pred):
    BOX = len(ANCHORS)
    NORM_W, NORM_H = GRID_W * 32, GRID_H * 32      # Scale back to get image input size
    SCALE_NOOB, SCALE_CONF, SCALE_COOR, SCALE_PROB = 0.5, 5.0, 5.0, 1.0

    y_pred = K.reshape(y_pred, [-1, GRID_W, GRID_H, BOX, N_CLASSES + 5])
#     y_true = K.reshape(y_true, [-1, GRID_W, GRID_H, BOX, N_CLASSES + 5])

    ### Adjust prediction
    # adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[...,:2])
    
    # adjust w and h
    pred_box_wh = tf.exp(y_pred[...,2:4]) * np.reshape(ANCHORS, [1,1,1, BOX, 2])
    pred_box_wh = tf.sqrt(pred_box_wh / np.reshape([float(GRID_W), float(GRID_H)], [1,1,1,1,2]))
    
    # adjust confidence
    pred_box_conf = tf.expand_dims(tf.sigmoid(y_pred[..., 4]), -1)
    
    # adjust probability
    pred_box_prob = tf.nn.softmax(y_pred[:, :, :, :, 5:])
    
    ### Adjust ground truth
    # adjust x and y
    center_xy   = y_true[...,:2]
    center_xy   = center_xy / np.reshape([(float(NORM_W)/GRID_W), (float(NORM_H)/GRID_H)], [1,1,1,1,2])
    true_box_xy = center_xy - tf.floor(center_xy)
    
    # adjust w and h
    true_box_wh = y_true[...,2:4]
    true_box_wh = tf.sqrt(true_box_wh / np.reshape([float(NORM_W), float(NORM_H)], [1,1,1,1,2]))
    
    # adjust confidence
    pred_tem_wh = tf.pow(pred_box_wh, 2) * np.reshape([GRID_W, GRID_H], [1,1,1,1,2])
    pred_box_area = pred_tem_wh[..., 0] * pred_tem_wh[..., 1]
    pred_box_ul = pred_box_xy - 0.5 * pred_tem_wh
    pred_box_bd = pred_box_xy + 0.5 * pred_tem_wh
    
    true_tem_wh = tf.pow(true_box_wh, 2) * np.reshape([GRID_W, GRID_H], [1,1,1,1,2])
    true_box_area = true_tem_wh[...,0] * true_tem_wh[...,1]
    true_box_ul = true_box_xy - 0.5 * true_tem_wh
    true_box_bd = true_box_xy + 0.5 * true_tem_wh
    
    intersect_ul = tf.maximum(pred_box_ul, true_box_ul) 
    intersect_br = tf.minimum(pred_box_bd, true_box_bd)
    intersect_wh = intersect_br - intersect_ul
    intersect_wh = tf.maximum(intersect_wh, 0.0)
    intersect_area = intersect_wh[...,0] * intersect_wh[...,1]
    
    iou = tf.truediv(intersect_area, true_box_area + pred_box_area - intersect_area)
    best_box = tf.equal(iou, tf.reduce_max(iou, [3], True)) 
    best_box = tf.to_float(best_box)
    true_box_conf = tf.expand_dims(best_box * y_true[...,4], -1)
    
    # adjust confidence
    true_box_prob = y_true[...,5:]
    
    y_true = tf.concat([true_box_xy, true_box_wh, true_box_conf, true_box_prob], 4)
    y_pred = tf.concat([pred_box_xy, pred_box_wh, pred_box_conf, pred_box_prob], 4)

    ### Compute the weights
    weight_coor = tf.concat(4 * [true_box_conf], 4)
    weight_coor = SCALE_COOR * weight_coor
    weight_conf = SCALE_NOOB * (1. - true_box_conf) + SCALE_CONF * true_box_conf
    weight_prob = tf.concat(N_CLASSES * [true_box_conf], 4) 
    weight_prob = SCALE_PROB * weight_prob 
    
    weight = tf.concat([weight_coor, weight_conf, weight_prob], 4)
    
    ### Finalize the loss
    loss = tf.pow(y_pred - y_true, 2)
    loss = loss * weight
    loss = weight
    loss = tf.reshape(loss, [-1, GRID_W*GRID_H*BOX*(4 + 1 + N_CLASSES)])
    loss = tf.reduce_sum(loss, 1)
    loss = .5 * tf.reduce_mean(loss)
    return loss

In [10]:
model.compile(optimizer= Adam(LEARN_RATE),
              loss     = custom_loss)
hist =  model.fit_generator(generator       = train_data_gen, 
                            steps_per_epoch = len(x_train) / BATCH_SIZE, 
                            epochs          = 10, 
                            workers=1, pickle_safe=False, verbose=1)

model.save_weights('darknet_weights.h5')

ValueError: Shape must be rank 5 but is rank 4 for 'concat_7' (op: 'ConcatV2') with input shapes: [1,?,?,?,2], [1,?,?,?,2], [?,40,30,5,1], [?,?,?,?], [].

## Multi-GPUs Training - Data Parallelism Approach

* Each GPU will have a copy of the model
* During training time, mean of all gradidents from each GPU will be calculated to update the model
<img style="width:40%" src="https://www.tensorflow.org/images/Parallelism.png">

### Visualize training process using Tensorboard
Open `http://<public-dns>:6006`

In [ ]:
import cv2
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

def extract_sign(img, bbox, output_size=(32, 32)):
    xc, yc, w, h = bbox.x, bbox.y, bbox.w, bbox.h
    x1   = int(xc - w/2)
    y1   = int(yc - h/2)
    x2   = int(xc + w/2)
    y2   = int(yc + h/2)
    roi = img[y1:y2, x1:x2]
    roi = cv2.resize(roi, output_size)
    return roi
x_train, y_train = shuffle(x_train, y_train)
fig = plt.figure(figsize=(17, 8))
for i, label in enumerate(labels):
    ax           = fig.add_subplot(4, 8, 1 + i, xticks=[], yticks=[])
    idx          = np.where(y_train[:, 1] == label)[0][0]
    img          = cv2.cvtColor(cv2.imread(x_train[idx]), cv2.COLOR_BGR2RGB)
    box          = y_train[idx][0]
    sign_only    = extract_sign(img, box, (32, 32)) #  just extract the sign
    ax.set_title(label)
    plt.imshow(sign_only)
plt.show()

In [ ]:
## Count frequencies of each traffic sign
labels, frequencies = np.unique(y_train[:,1 ], return_counts=True)

plt.figure(figsize=(20, 5))
x = np.arange(len(labels))
plt.xticks(x, labels, rotation=70)
plt.bar(x, frequencies)